This file: **only** involves calculating storage-needed weights, not for plotting
- 1. Generating distance matrix from raw transcription (for Yubao), and select parts of dialects


In [123]:
import pickle
import numpy as np
import os

data4_dir, data4_dir_matrix = 'Data4/transcription_areas.pkl', 'Data4/distance_matrices.npz'

def list_available_data():
    """
    列出可用的数据集及其包含的特征
    """
    available_data = {
        'Data4': {
            'description': '中国语言资源保护工程',
            'features': [
                'word_name', 'area', 'slice', 'slices', 'coords',
                'initial', 'final', 'tone',
                'initials_distance', 'finals_distance', 'tones_distance', 'overall_distance'
            ]
        }
        # 可以添加其他数据集
    }
    return available_data

def load_feats(name, features=None):
    """
    加载指定数据集的指定特征

    Args:
        name (str): 数据集名称 (e.g., 'Data4')
        features (list, optional): 需要加载的特征列表.
                                     如果为None，则加载所有特征. Defaults to None.

    Returns:
        dict: 包含请求特征的字典
    """
    loaded_data = {}

    if name == 'Data4':
        all_features = [
            'word_name', 'area', 'slice', 'slices', 'coords',
            'initial', 'final', 'tone',
            'initials_distance', 'finals_distance', 'tones_distance', 'overall_distance'
        ]
        if features is None:
            features_to_load = all_features
        else:
            features_to_load = [f for f in features if f in all_features]
            if len(features_to_load) != len(features):
                print(f"Warning: Some requested features for {name} are not available.")

        # 加载 pkl 文件中的数据
        pkl_features = ['word_name', 'area', 'slice', 'slices', 'coords', 'initial', 'final', 'tone']
        if any(f in features_to_load for f in pkl_features):
            try:
                with open(data4_dir, 'rb') as f:
                    data_dict = pickle.load(f)
                if 'initial' in features_to_load:
                     loaded_data['initial'] = data_dict.get('initial')
                if 'final' in features_to_load:
                     loaded_data['final'] = data_dict.get('final')
                if 'tone' in features_to_load:
                     loaded_data['tone'] = data_dict.get('tone')
                if 'word_name' in features_to_load:
                    loaded_data['word_name'] = data_dict.get('word_name')
                if 'area' in features_to_load:
                    loaded_data['area'] = data_dict.get('area')
                if 'slice' in features_to_load:
                    loaded_data['slice'] = data_dict.get('slice')
                if 'slices' in features_to_load:
                    loaded_data['slices'] = data_dict.get('slices')
                if 'coords' in features_to_load:
                    loaded_data['coords'] = data_dict.get('coords')

            except FileNotFoundError:
                print(f"Error: {data4_dir} not found.")
            except Exception as e:
                print(f"Error loading {data4_dir}: {e}")


        # 加载 npz 文件中的数据
        npz_features = ['initials_distance', 'finals_distance', 'tones_distance', 'overall_distance']
        if any(f in features_to_load for f in npz_features):
            try:
                loaded_npz = np.load(data4_dir_matrix)
                if 'initials_distance' in features_to_load:
                    loaded_data['initials_distance'] = loaded_npz.get('initials')
                if 'finals_distance' in features_to_load:
                    loaded_data['finals_distance'] = loaded_npz.get('finals')
                if 'tones_distance' in features_to_load:
                    loaded_data['tones_distance'] = loaded_npz.get('tones')
                if 'overall_distance' in features_to_load:
                     loaded_data['overall_distance'] = loaded_npz.get('overall')
                loaded_npz.close() # Close the npz file

            except FileNotFoundError:
                print(f"Error: {data4_dir_matrix} not found.")
            except Exception as e:
                print(f"Error loading {data4_dir_matrix}: {e}")

    else:
        print(f"Error: Dataset '{name}' not found.")

    return loaded_data

# **Loading**

In [14]:
import numpy as np
import pickle
import os # 导入 os 库用于路径拼接

# --- 数据文件路径定义 (请根据你的实际文件位置修改这些路径) ---
# 假设你的数据文件存放在项目根目录下的 data/Data4 文件夹内
# 你可能需要根据实际情况调整这些路径
BASE_DATA3_DIR, BASE_DATA4_DIR, BASE_DATA2_DIR, BASE_DATA1_DIR = 'Data3', 'Data4', 'Data2', 'Data1' # 数据文件所在的基准目录

# Data4 的原始转写和元数据 pkl 文件
data4_raw_data_path = os.path.join(BASE_DATA4_DIR, 'transcription_areas.pkl') 
data4_distance_matrix_path = os.path.join(BASE_DATA4_DIR, 'distance_matrices.npz')
data4_processed_info_path = os.path.join(BASE_DATA4_DIR, 'processed_info.pkl')
# -------------------------------------------------------------
data3_distance_matrix_path = os.path.join(BASE_DATA3_DIR, 'distance_matrices.npz')
data3_processed_info_path = os.path.join(BASE_DATA3_DIR, 'info.npz')
# -------------------------------------------------------------
data1_processed_info_path = os.path.join(BASE_DATA1_DIR, 'data.pkl')
# -------------------------------------------------------------
data2_mfcc_dialect_mean_path = os.path.join(BASE_DATA2_DIR, 'dialect_mean_features.npz') 
data2_mfcc_dialect_slice_path = os.path.join(BASE_DATA2_DIR, 'dialect_slice_mean_features.npz') 
data2_mfcc_dialect_gmm_ivector_path = os.path.join(BASE_DATA2_DIR, 'mfcc_gmm_ivectordialect.npz') 


def load_feats(name, type=None, features=None):
    """
    加载指定数据集的指定类型或指定特征的数据。

    Args:
        name (str): 数据集名称 (e.g., 'Data4')
        type (str, optional): 需要加载的数据类型 (e.g., 'raw', 'distance_matrices').
                              如果指定 type，函数会加载该类型下的预定义特征。
                              如果 type 为 None，则必须通过 features 参数指定要加载的特征。
        features (list, optional): 需要加载的特征列表.
                                     如果 type 已指定，features 可用于过滤该类型下的特征。
                                     如果 type 为 None，则加载此列表中指定的特征。
                                     Defaults to None.

    Returns:
        dict: 包含请求特征的字典，键为特征名，值为对应的数据。
              如果加载失败或未找到数据集/类型，返回空字典或 None。
    """
    loaded_data = {}

    # --- 定义不同数据集和类型下的特征列表和文件路径 ---
    # 这个字典定义了每个数据集名称下，不同类型对应哪些预定义特征以及从哪个文件加载
    DATASET_CONFIG = {
        'Data4': {
            'raw': {
                'file': data4_raw_data_path,
                'pkl_keys': ['word_name', 'area', 'slice', 'slices', 'coords', 'initial', 'final', 'tone'],
                'loader': 'pickle' # 指定加载方式
            },
            'distance_matrices': {
                'file': data4_distance_matrix_path,
                'npz_keys': ['initials', 'finals', 'tones', 'overall'], # npz 文件中的键名
                'output_keys': ['initials_distance', 'finals_distance', 'tones_distance', 'overall_distance'], # 输出字典中的键名
                'loader': 'numpy_npz' # 指定加载方式
            },
            'info': {
                'file': data4_processed_info_path,
                # 处理后信息文件的键名和输出键名一致
                'pkl_keys': ['areas', 'slice', 'slices', 'coords', 'word_names'], # 注意这里的键名与保存时字典的键名对应
                'loader': 'pickle'}
            # 可以继续添加其他 type...
            # 'another_type': {...}
        },
        'Data3': {
            'distance_matrices': {
                'file': data3_distance_matrix_path,
                'npz_keys': ['lexicon', 'phonology', 'syntax', 'overall'], # npz 文件中的键名
                'output_keys': ['lexicon_distance', 'phonology_distance', 'syntax_distance', 'overall_distance'], # 输出字典中的键名
                'loader': 'numpy_npz' # 指定加载方式
            },
            'info': {
                'file': data3_processed_info_path,
                # 处理后信息文件的键名和输出键名一致
                'npz_keys': ['coords'], # 注意这里的键名与保存时字典的键名对应
                'output_keys': ['coords'],
                'loader': 'numpy_npz'}
            # 可以继续添加其他 type...
            # 'another_type': {...}
        },
        'Data1':{
            'info': {
                'file': data1_processed_info_path,
                # 处理后信息文件的键名和输出键名一致
                'pkl_keys': ['word', 'initial', 'final_1', 'final_2', 'final_3', 'tone'], # 注意这里的键名与保存时字典的键名对应
                'loader': 'pickle'}
        },
        'Data2':{
            'mfcc_dialect_mean': {
                'file': data2_mfcc_dialect_mean_path,
                'npz_keys': ['features', 'dialect_names'], # npz 文件中的键名
                'output_keys': ['features', 'names'], # 输出字典中的键名
                'loader': 'numpy_npz' # 指定加载方式
            },
            'mfcc_slice_mean': {
                'file': data2_mfcc_dialect_slice_path,
                'npz_keys': ['features', 'slice_names'], # npz 文件中的键名
                'output_keys': ['features', 'names'], # 输出字典中的键名
                'loader': 'numpy_npz' # 指定加载方式
            },  
            'mfcc_dialect_gmm_ivector': {
                'file': data2_mfcc_dialect_gmm_ivector_path,
                'npz_keys': ['features', 'dialect_names'], # npz 文件中的键名
                'output_keys': ['features', 'names'], # 输出字典中的键名
                'loader': 'numpy_npz' # 指定加载方式
            },
        }
    }

    # --- 检查数据集名称是否存在 ---
    if name not in DATASET_CONFIG:
        print(f"错误: 数据集 '{name}' 的配置不存在。")
        return {} # 返回空字典表示失败

    dataset_config = DATASET_CONFIG[name]

    # --- 根据 type 或 features 确定要加载的特征和文件 ---
    features_to_load_final = [] # 最终确定要加载的特征列表
    file_to_load = None
    loader_type = None
    source_keys = {} # 记录源文件中的键名到目标输出键名的映射

    if type:
        if type not in dataset_config:
            print(f"错误: 数据集 '{name}' 不支持类型 '{type}'。")
            return {}

        type_config = dataset_config[type]
        file_to_load = type_config.get('file')
        loader_type = type_config.get('loader')

        if loader_type == 'pickle':
            all_type_features = type_config.get('pkl_keys', [])
            # pickle 加载是直接从字典取，源键和输出键一致
            source_keys = {k: k for k in all_type_features}
        elif loader_type == 'numpy_npz':
            all_type_features = type_config.get('output_keys', [])
            # npz 加载需要处理源键到输出键的映射
            npz_keys = type_config.get('npz_keys', [])
            output_keys = type_config.get('output_keys', [])
            if len(npz_keys) == len(output_keys):
                 source_keys = dict(zip(output_keys, npz_keys)) # 存储 输出键 -> 源键 映射
            else:
                 print(f"配置错误: 数据集 '{name}', 类型 '{type}' 的 npz_keys 和 output_keys 数量不匹配。")
                 return {}
        else:
             print(f"配置错误: 数据集 '{name}', 类型 '{type}' 指定了未知的 loader '{loader_type}'。")
             return {}


        if features is None:
            # 如果没有指定 features，则加载该 type 下的所有预定义特征
            features_to_load_final = all_type_features
        else:
            # 如果指定了 features，则加载该 type 下 features 中包含的特征
            features_to_load_final = [f for f in features if f in all_type_features]
            if len(features_to_load_final) != len(features):
                # 检查用户请求的 features 中是否有不属于该 type 的
                not_available = [f for f in features if f not in all_type_features]
                print(f"警告: 请求的特征 {not_available} 不属于数据集 '{name}' 的类型 '{type}'，将被忽略。")

    elif features is not None:
         # 如果 type 为 None 但指定了 features (旧的使用方式，可以保留兼容性或弃用)
         # 为了新设计清晰，建议要求必须指定 type
         print("错误: 未指定数据加载类型 (type)，请指定如 type='raw'。")
         return {}
         # 以下是保留旧 features 用法的代码，如果需要兼容可以启用：
         # print("警告: 未指定数据类型 (type)，尝试按特征列表加载 (旧模式)。")
         # # 需要遍历所有 type 才能找到哪些特征在哪里，比较复杂，不推荐。
         # # 更好的方式是：要求用户必须指定 type。
         # pass # 在新设计中不推荐无 type 加载

    else:
        # type 和 features 都为 None
        print("错误: 既未指定数据加载类型 (type)，也未指定要加载的特征列表 (features)。")
        return {}


    # --- 执行数据加载 ---
    if not file_to_load:
         print("内部错误: 未能确定要加载的文件路径。")
         return {}

    if not features_to_load_final:
        print(f"未找到需要加载的特征列表，请检查 type 或 features 参数。")
        return {}

    print(f"正在从文件 '{file_to_load}' 加载数据...")
    print(f"计划加载的特征: {features_to_load_final}")

    try:
        if loader_type == 'pickle':
            with open(file_to_load, 'rb') as f:
                data_dict = pickle.load(f)

            for feature_name in features_to_load_final:
                # 从加载的字典中按特征名获取数据
                # 使用 .get() 避免 KeyError 如果文件中的确缺少该特征
                if feature_name in data_dict:
                     loaded_data[feature_name] = data_dict[feature_name]
                else:
                     print(f"警告: 在文件 '{file_to_load}' 中未找到特征 '{feature_name}'。")

        elif loader_type == 'numpy_npz':
            loaded_npz = np.load(file_to_load)
            for output_key in features_to_load_final:
                 source_key = source_keys.get(output_key) # 获取 npz 文件中的对应键
                 if source_key and source_key in loaded_npz:
                     loaded_data[output_key] = loaded_npz[source_key]
                 else:
                     print(f"警告: 在文件 '{file_to_load}' 中未找到特征 '{output_key}' (查找键 '{source_key}')。")
            loaded_npz.close() # 关闭 npz 文件句柄

    except FileNotFoundError:
        print(f"错误: 数据文件 '{file_to_load}' 未找到。请检查路径设置。")
        return {} # 返回空字典表示失败
    except Exception as e:
        print(f"错误加载文件 '{file_to_load}': {e}")
        return {} # 返回空字典表示失败

    print(f"成功加载 {len(loaded_data)} 个特征。")
    return loaded_data


In [66]:
dict_ = load_feats(name='Data4', type='raw')

正在从文件 'Data4/transcription_areas.pkl' 加载数据...
计划加载的特征: ['word_name', 'area', 'slice', 'slices', 'coords', 'initial', 'final', 'tone']
成功加载 8 个特征。


# 1. Data4 

In [ ]:
dict = load_feats(name='Data4', type='raw')
# ['initial', 'final', 'tone', 'word_name', 'area', 'slice', 'slices', 'coords']
initials, finals, tones, word_names, areas, slice, slices, coords = dict['initial'], dict['final'], dict['tone'], dict['word_name'], dict['area'], dict['slice'], dict['slices'], dict['coords']

## 1.1 Calculate and Save Distance Matrix
We first calculate the ratio of missing values of different dialects and features, any only select those with non-missing value above 70%

### 1.1.1 Ratio of Missing Values

In [27]:
import numpy as np

def ratio_missing_transcription(numpy_array, missing_value='MISSING', threshold_percentage=30):
    """
    分析 NumPy 数组中指定缺失值的比例，找出超过阈值的行和列，
    并返回缺失比例最高的各自 Top 10。输出结果使用标准 Python 数字类型。

    Args:
        numpy_array (np.ndarray): 输入的 NumPy 数组，应为二维。
        missing_value (str): 表示缺失值的字符串或值，默认为 'MISSING'。
        threshold_percentage (int or float): 缺失值比例阈值 (0-100)，
                                            超过此比例的行/列会被单独列出。默认为 30。

    Returns:
        dict: 包含分析结果的字典，键值如下：
            'rows_over_threshold': 缺失比例超过阈值的行索引列表 (Python int)。
            'cols_over_threshold': 缺失比例超过阈值的列索引列表 (Python int)。
            'top10_rows': 缺失比例最高的 Top 10 行 (索引, 比例) 列表。
                           格式为 [(行索引: Python int, 缺失百分比: Python float), ...]。
            'top10_cols': 缺失比例最高的 Top 10 列 (索引, 比例) 列表。
                           格式为 [(列索引: Python int, 缺失百分比: Python float), ...]。
        None: 如果输入不是二维 NumPy 数组，则返回 None 并打印错误信息。
    """
    if not isinstance(numpy_array, np.ndarray) or numpy_array.ndim != 2:
        print("错误：输入必须是一个二维 NumPy 数组。")
        return None

    n_rows, n_cols = numpy_array.shape
    print(f"正在分析一个 {n_rows} 行, {n_cols} 列的数组...")
    print(f"缺失值标记为: '{missing_value}'")
    print(f"缺失比例阈值为: {threshold_percentage}%")

    # 创建一个布尔掩码，标记出所有缺失值的位置
    missing_mask = (numpy_array == missing_value)

    # --- 计算每行的缺失比例 ---
    row_missing_counts = np.sum(missing_mask, axis=1)
    row_missing_percentages = (row_missing_counts / n_cols) * 100

    # --- 计算每列的缺失比例 ---
    col_missing_counts = np.sum(missing_mask, axis=0)
    col_missing_percentages = (col_missing_counts / n_rows) * 100

    print("缺失比例计算完成。")

    # --- 找出比例超过阈值的行和列 ---
    # 转换为标准 Python int 列表
    rows_over_threshold_indices = [int(i) for i in np.where(row_missing_percentages > threshold_percentage)[0]]
    cols_over_threshold_indices = [int(i) for i in np.where(col_missing_percentages > threshold_percentage)[0]]

    print(f"发现 {len(rows_over_threshold_indices)} 行的缺失比例超过 {threshold_percentage}%。")
    print(f"发现 {len(cols_over_threshold_indices)} 列的缺失比例超过 {threshold_percentage}%。")

    # --- 找出缺失比例最高的 Top 10 行 ---
    top10_row_indices = np.argsort(-row_missing_percentages)[:10]
    top10_row_percentages = row_missing_percentages[top10_row_indices]
    # 转换为标准 Python int 和 float 元组列表
    top10_rows_result = [(int(idx), float(round(pct, 2))) for idx, pct in zip(top10_row_indices, top10_row_percentages)]

    # --- 找出缺失比例最高的 Top 10 列 ---
    top10_col_indices = np.argsort(-col_missing_percentages)[:10]
    top10_col_percentages = col_missing_percentages[top10_col_indices]
    # 转换为标准 Python int 和 float 元组列表
    top10_cols_result = [(int(idx), float(round(pct, 2))) for idx, pct in zip(top10_col_indices, top10_col_percentages)]

    print("Top 10 缺失比例行/列计算完成。")

    # --- 构建结果字典 ---
    results = {
        'rows_over_threshold': rows_over_threshold_indices,
        'cols_over_threshold': cols_over_threshold_indices,
        'top10_rows': top10_rows_result,
        'top10_cols': top10_cols_result,
    }

    return results

In [48]:
results_initials = ratio_missing_transcription(initials, missing_value='MISSING', threshold_percentage=20)

正在分析一个 1289 行, 999 列的数组...
缺失值标记为: 'MISSING'
缺失比例阈值为: 20%
缺失比例计算完成。
发现 6 行的缺失比例超过 20%。
发现 2 列的缺失比例超过 20%。
Top 10 缺失比例行/列计算完成。


In [49]:
results_finals = ratio_missing_transcription(finals, missing_value='MISSING', threshold_percentage=20)

正在分析一个 1289 行, 999 列的数组...
缺失值标记为: 'MISSING'
缺失比例阈值为: 20%
缺失比例计算完成。
发现 186 行的缺失比例超过 20%。
发现 84 列的缺失比例超过 20%。
Top 10 缺失比例行/列计算完成。


In [50]:
results_tones = ratio_missing_transcription(tones, missing_value='MISSING', threshold_percentage=20)

正在分析一个 1289 行, 999 列的数组...
缺失值标记为: 'MISSING'
缺失比例阈值为: 20%
缺失比例计算完成。
发现 23 行的缺失比例超过 20%。
发现 2 列的缺失比例超过 20%。
Top 10 缺失比例行/列计算完成。


In [51]:
# first operate dialects 
all_row_indices = results_initials['rows_over_threshold'] + results_finals['rows_over_threshold'] + results_tones['rows_over_threshold']
unique_row_indices_set = set(all_row_indices)
unique_row_indices_list = list(unique_row_indices_set)
print(len(unique_row_indices_list), unique_row_indices_list)

all_col_indices = results_initials['cols_over_threshold'] + results_finals['cols_over_threshold'] + results_tones['cols_over_threshold']
unique_col_indices_set = set(all_col_indices)
unique_col_indices_list = list(unique_col_indices_set)
print(len(unique_col_indices_list), unique_col_indices_list)

205 [7, 520, 524, 14, 1039, 17, 533, 535, 541, 31, 550, 551, 40, 41, 553, 555, 556, 559, 561, 562, 563, 564, 565, 568, 574, 575, 1087, 580, 583, 585, 586, 587, 76, 588, 81, 1105, 83, 595, 596, 87, 599, 604, 1117, 94, 606, 1124, 101, 104, 108, 111, 112, 113, 114, 115, 1138, 118, 1144, 121, 1147, 125, 131, 132, 644, 134, 1155, 138, 141, 144, 145, 146, 656, 148, 149, 150, 151, 153, 1178, 669, 158, 159, 1185, 673, 165, 677, 168, 1193, 170, 171, 172, 174, 1199, 176, 1200, 178, 692, 1205, 1211, 187, 189, 1213, 192, 705, 196, 197, 710, 1221, 1224, 1226, 204, 1230, 719, 1231, 724, 734, 1248, 1250, 1251, 1259, 1263, 1264, 1273, 1274, 254, 1282, 259, 1284, 1286, 1288, 277, 286, 288, 801, 296, 331, 848, 850, 853, 345, 347, 859, 864, 865, 873, 878, 372, 886, 890, 378, 380, 386, 387, 904, 397, 398, 911, 913, 405, 408, 409, 412, 928, 932, 422, 424, 425, 427, 428, 431, 432, 434, 439, 441, 958, 961, 450, 963, 964, 965, 969, 459, 460, 971, 974, 972, 973, 976, 982, 983, 473, 1016, 988, 476, 477, 479, 48

In [52]:
import numpy as np

def remove_rows_and_cols(numpy_array, rows_to_delete, cols_to_delete):
    """
    从一个二维 NumPy 数组中删除指定的行和列。

    Args:
        numpy_array (np.ndarray): 输入的二维 NumPy 数组。
        rows_to_delete (list or array_like): 要删除的行的索引列表或数组。
        cols_to_delete (list or array_like): 要删除的列的索引列表或数组。

    Returns:
        np.ndarray: 删除指定行和列后的新的 NumPy 数组。
        None: 如果输入不是二维 NumPy 数组，则返回 None 并打印错误信息。
    """
    if not isinstance(numpy_array, np.ndarray) or numpy_array.ndim != 2:
        print("错误：输入必须是一个二维 NumPy 数组。")
        return None

    print(f"原始数组维度: {numpy_array.shape}")
    print(f"将删除 {len(rows_to_delete)} 行和 {len(cols_to_delete)} 列...")

    # 使用 np.delete 删除行
    # axis=0 表示删除行
    # 返回一个新的数组，原始数组不变
    array_after_rows_deleted = np.delete(numpy_array, rows_to_delete, axis=0)

    # 使用 np.delete 删除列
    # 注意：这里是在删除行后的新数组上操作
    # axis=1 表示删除列
    array_after_cols_deleted = np.delete(array_after_rows_deleted, cols_to_delete, axis=1)

    print(f"删除后的数组维度: {array_after_cols_deleted.shape}")

    return array_after_cols_deleted

In [56]:
processed_initials = remove_rows_and_cols(initials, unique_row_indices_list, unique_col_indices_list)
processed_finals = remove_rows_and_cols(finals, unique_row_indices_list, unique_col_indices_list)
processed_tones = remove_rows_and_cols(tones, unique_row_indices_list, unique_col_indices_list)

原始数组维度: (1289, 999)
将删除 205 行和 84 列...
删除后的数组维度: (1084, 915)
原始数组维度: (1289, 999)
将删除 205 行和 84 列...
删除后的数组维度: (1084, 915)
原始数组维度: (1289, 999)
将删除 205 行和 84 列...
删除后的数组维度: (1084, 915)


### 1.1.2 Distance Calculation

In [57]:
import numpy as np
# Optional: Import tqdm for a progress bar, useful for large datasets
# from tqdm.auto import tqdm # pip install tqdm

def cal_distance(features, missing_value='MISSING'):
    """
    Calculate the distance matrix between dialects based on feature differences.

    The distance d(i, j) is the number of features where dialects i and j differ,
    divided by the number of features where *neither* dialect i nor j has a
    missing value.

    Args:
        features (np.ndarray): 2D array [n_dialects, n_features], dtype=object.
                               Contains feature values (usually strings).
        missing_value (str): Indicator of missing values (e.g., 'MISSING', 'Ǿ').
                               Features with this value are ignored in pairwise comparisons.

    Returns:
        np.ndarray: Distance matrix [n_dialects, n_dialects], dtype=float.
                    dist_matrix[i, j] is the calculated distance.
                    Returns np.nan if a pair of dialects has no comparable features
                    (i.e., for every feature, at least one has a missing value).
    """
    if not isinstance(features, np.ndarray) or features.ndim != 2:
        raise ValueError("Input 'features' must be a 2D NumPy array.")

    n_dialects, n_features = features.shape
    print(f"Calculating distance matrix for {n_dialects} dialects and {n_features} features...")
    print(f"Using '{missing_value}' as the missing value indicator.")

    # Initialize distance matrix - using np.nan allows easy identification
    # of pairs with no comparable features. Diagonal will be set to 0.
    dist_matrix = np.full((n_dialects, n_dialects), np.nan, dtype=float)

    # Pre-calculate a boolean mask indicating where features are *not* missing
    # This avoids repeated comparisons with missing_value inside the loops
    is_valid = (features != missing_value)
    print("Pre-calculated validity mask.")

    # --- Iterate through all unique pairs of dialects (i, j) ---
    # Using tqdm here provides a nice progress bar: range(n_dialects) -> tqdm(range(n_dialects))
    # for i in tqdm(range(n_dialects), desc="Calculating distances"):
    for i in range(n_dialects):
        # Optional: Print progress every N dialects
        if (i + 1) % 100 == 0:
            print(f"  Processed {i+1}/{n_dialects} dialects...")

        # Distance from a dialect to itself is 0
        dist_matrix[i, i] = 0.0

        # Get data and validity mask for dialect i once
        features_i = features[i]
        is_valid_i = is_valid[i]

        # Compare dialect i with dialects j where j > i
        for j in range(i + 1, n_dialects):
            features_j = features[j]
            is_valid_j = is_valid[j]

            # 1. Find features where *both* dialects have valid data
            #    (element-wise AND)
            valid_comparison_mask = is_valid_i & is_valid_j

            # 2. Count how many features are valid for this pair
            num_valid_features = np.sum(valid_comparison_mask)

            # 3. Handle case where there are no features to compare
            if num_valid_features == 0:
                # Distance is undefined, leave as np.nan
                # dist_matrix[i, j] = np.nan # Already initialized to NaN
                dist_matrix[j, i] = np.nan # Ensure symmetry for NaN too
                continue # Move to the next pair (j)

            # 4. Compare feature values *only* where both are valid
            #    First, find all differing features between the full rows
            feature_differs = (features_i != features_j)
            #    Then, select differences only where the comparison was valid
            valid_differences_mask = feature_differs & valid_comparison_mask

            # 5. Count the number of valid differences
            num_differences = np.sum(valid_differences_mask)

            # 6. Calculate the distance
            distance = num_differences / num_valid_features

            # 7. Store the distance (and maintain symmetry)
            dist_matrix[i, j] = distance
            dist_matrix[j, i] = distance

    print("Distance matrix calculation finished.")
    return dist_matrix

In [ ]:
initials_distance = cal_distance(processed_initials)
finals_distance = cal_distance(processed_finals)
tones_distance = cal_distance(processed_tones)

In [59]:
overall_distance = (initials_distance + finals_distance + tones_distance) / 3
output_filename = 'Data4/distance_matrices.npz'
np.savez_compressed(
    output_filename,
    initials=initials_distance,  # 在 .npz 文件中，这个矩阵被称为 'initials'
    finals=finals_distance,    # 这个被称为 'finals'
    tones=tones_distance,       # 这个被称为 'tones'
    overall=overall_distance  # 如果需要，可以添加整体距离矩阵
)
print(f"成功将距离矩阵保存到: {output_filename}")

成功将距离矩阵保存到: Data4/distance_matrices.npz


In [60]:
output_filename = 'Data4/distance_matrices.npz'
print(f"\n尝试加载文件: {output_filename}")
loaded_data = np.load(output_filename)

# 检查文件中有哪些数组
print(f"文件中包含的数组名称: {list(loaded_data.keys())}")

# 加载单个数组
loaded_initials = loaded_data['initials']
loaded_finals = loaded_data['finals']
loaded_tones = loaded_data['tones']
overall_distance = loaded_data['overall']

print(f"加载的 'initials' 矩阵形状: {loaded_initials.shape}")
print(f"加载的 'finals' 矩阵形状: {loaded_finals.shape}")
print(f"加载的 'tones' 矩阵形状: {loaded_tones.shape}")

# 不要忘记关闭文件（虽然对于 np.load 通常不是严格必需的，但好习惯）
loaded_data.close()


尝试加载文件: Data4/distance_matrices.npz
文件中包含的数组名称: ['initials', 'finals', 'tones', 'overall']
加载的 'initials' 矩阵形状: (1084, 1084)
加载的 'finals' 矩阵形状: (1084, 1084)
加载的 'tones' 矩阵形状: (1084, 1084)


In [98]:
import numpy as np
import typing

def get_missing_percentage(numpy_matrix: np.ndarray, missing_value: typing.Any = 'MISSING') -> float:
    """
    计算 NumPy 数组中指定缺失值（默认为 'MISSING'）的元素比例。

    Args:
        numpy_matrix (np.ndarray): 输入的 NumPy 数组（可以是任意维度）。
        missing_value (Any): 用来表示缺失值的值，默认为 'MISSING' 字符串。

    Returns:
        float: 元素中缺失值的百分比（0.0 到 100.0 之间）。
               如果输入数组为空，返回 0.0。
               如果输入不是 NumPy 数组，打印错误并返回 0.0。
    """
    if not isinstance(numpy_matrix, np.ndarray):
        print("错误: 输入必须是一个 NumPy 数组。")
        return 0.0

    total_elements = numpy_matrix.size

    # 检查数组是否为空，避免除以零
    if total_elements == 0:
        print("警告: 输入数组为空，缺失比例为 0%。")
        return 0.0

    # 创建布尔掩码，True 表示元素等于 missing_value
    missing_mask = (numpy_matrix == missing_value)

    # 统计缺失值的数量 (True 会被当作 1 求和)
    missing_count = np.sum(missing_mask)

    # 计算并返回百分比
    percentage = (missing_count / total_elements) * 100.0
    print(f"缺失值比例: {percentage:.2f}%")
    return percentage

In [100]:
get_missing_percentage(processed_initials, missing_value='MISSING')
get_missing_percentage(processed_finals, missing_value='MISSING') 
get_missing_percentage(processed_tones, missing_value='MISSING')

缺失值比例: 1.01%
缺失值比例: 5.98%
缺失值比例: 1.22%


np.float64(1.2160990462363641)

### 1.1.3 Processing Other Information

In [86]:
import numpy as np
import typing

def remove_elements_or_rows_by_indices(data: typing.Union[list, np.ndarray], indices_to_delete: typing.List[int]) -> list:
    """
    从列表、一维 NumPy 数组或二维 NumPy 数组中删除指定索引位置的元素/行。

    Args:
        data (list or np.ndarray): 输入的数据，可以是 Python 列表、一维或二维 NumPy 数组。
        indices_to_delete (list): 包含要删除元素的索引（位置）或行的索引的列表（整数）。

    Returns:
        list: 删除指定元素/行后的结果，统一返回 Python 列表。
              对于原始输入是列表或一维 NumPy 数组，返回的是一个扁平的列表。
              对于原始输入是二维 NumPy 数组，返回的是一个列表的列表。
              如果指定的索引超出范围，超出范围的索引会被忽略。
              如果输入数据类型或 NumPy 数组维度不支持，返回 None 并打印错误。
    """
    if not isinstance(indices_to_delete, list):
        print("错误: 第二个输入（indices_to_delete）必须是 list 类型。")
        return None

    # 为了高效查找索引，转换为集合
    indices_to_delete_set = set(indices_to_delete)
    print(f"计划删除 {len(indices_to_delete)} 个索引。")

    result_list = None # 初始化为 None，表示尚未成功处理

    if isinstance(data, list):
        print("输入类型: Python 列表")
        print(f"原始列表长度: {len(data)}")
        # 使用列表推导式创建新列表，只包含索引不在删除集合中的元素
        result_list = [item for i, item in enumerate(data) if i not in indices_to_delete_set]
        print(f"删除后列表长度: {len(result_list)}")

    elif isinstance(data, np.ndarray):
        print("输入类型: NumPy 数组")
        
        array_after_deletion = None

        if data.ndim == 1:
            print(f"原始 NumPy 数组维度: {data.shape} (一维)")
            # 对于一维数组，np.delete(arr, obj, axis=0) 或 axis=None 都可以删除元素
            array_after_deletion = np.delete(data, list(indices_to_delete_set), axis=0)
            print(f"删除后 NumPy 数组维度: {array_after_deletion.shape}")
            # 将一维 NumPy 数组转换为 Python 列表
            result_list = array_after_deletion.tolist()

        elif data.ndim == 2:
            print(f"原始 NumPy 数组维度: {data.shape} (二维)")
            # 使用 np.delete 按行删除 (axis=0)
            array_after_deletion = np.delete(data, list(indices_to_delete_set), axis=0)
            print(f"删除后 NumPy 数组维度: {array_after_deletion.shape}")
            # 将 NumPy 二维数组转换为 Python 列表的列表
            result_list = array_after_deletion.tolist()
            
        else:
            print(f"错误: 输入的 NumPy 数组维度为 {data.ndim}，当前函数只支持一维或二维数组。")
            return None # 返回 None 表示处理失败

    else:
        print(f"错误: 不支持的数据类型 '{type(data)}'。输入必须是 list 或 np.ndarray。")
        return None # 返回 None 表示处理失败

    return result_list # 返回最终的列表结果

In [79]:
removed_word_names = remove_elements_or_rows_by_indices(word_names, unique_col_indices_list)

计划删除 84 个索引。
输入类型: Python 列表
原始列表长度: 999
删除后列表长度: 915


In [87]:
removed_areas, removed_slice, removed_slices, removed_coords = remove_elements_or_rows_by_indices(areas, unique_row_indices_list), remove_elements_or_rows_by_indices(slice, unique_row_indices_list), remove_elements_or_rows_by_indices(slices, unique_row_indices_list), remove_elements_or_rows_by_indices(coords, unique_row_indices_list)

计划删除 205 个索引。
输入类型: NumPy 数组
原始 NumPy 数组维度: (1289,) (一维)
删除后 NumPy 数组维度: (1084,)
计划删除 205 个索引。
输入类型: NumPy 数组
原始 NumPy 数组维度: (1289,) (一维)
删除后 NumPy 数组维度: (1084,)
计划删除 205 个索引。
输入类型: NumPy 数组
原始 NumPy 数组维度: (1289,) (一维)
删除后 NumPy 数组维度: (1084,)
计划删除 205 个索引。
输入类型: NumPy 数组
原始 NumPy 数组维度: (1289, 2) (二维)
删除后 NumPy 数组维度: (1084, 2)


In [90]:
removed_areas, removed_slice, removed_slices, removed_coords, removed_word_names

output_path = 'Data4/processed_info.pkl'
data_to_save = {
        'areas': removed_areas,
        'slice': removed_slice,
        'slices': removed_slices,
        'coords': removed_coords,
        'word_names': removed_word_names
    }

# 确保目录存在
output_dir = os.path.dirname(output_path)
if output_dir: # 只有当路径包含目录时才创建
    os.makedirs(output_dir, exist_ok=True)

try:
    with open(output_path, 'wb') as f:
        pickle.dump(data_to_save, f)
    print(f"成功将处理后信息保存到 '{output_path}'")
except Exception as e:
    print(f"错误保存文件 '{output_path}': {e}")

成功将处理后信息保存到 'Data4/processed_info.pkl'


# **Data 3**

In [112]:
import pandas as pd
data = pd.read_csv('Data3/LCAD_distmatrix_0.csv', index_col=0).to_numpy()  
data_lexicon = pd.read_csv('Data3/LCAD_distmatrix_lexicon.csv', index_col=0).to_numpy()  
data_phonology = pd.read_csv('Data3/LCAD_distmatrix_phonology.csv', index_col=0).to_numpy()  
data_syntax = pd.read_csv('Data3/LCAD_distmatrix_syntax.csv', index_col=0).to_numpy()  

In [114]:
locations = pd.read_csv('Data3/cn_tax.csv', index_col=0)
locs_df = locations[['long', 'lat']].to_numpy()

output_filename = 'Data3/info.npz'
np.savez_compressed(
    output_filename,
    coords=locs_df
)
print(f"成功将距离矩阵保存到: {output_filename}")

#后续补充了方言区和方言小片再加

成功将距离矩阵保存到: Data3/info.npz


In [113]:
output_filename = 'Data3/distance_matrices.npz'
np.savez_compressed(
    output_filename,
    lexicon=data_lexicon,  
    phonology=data_phonology,    
    syntax=data_syntax,       
    overall=data  
)
print(f"成功将距离矩阵保存到: {output_filename}")

成功将距离矩阵保存到: Data3/distance_matrices.npz


In [8]:
raw_data_dict = load_feats(name='Data3', type='distance_matrices')
info = load_feats(name='Data3', type='info')

正在从文件 'Data3/distance_matrices.npz' 加载数据...
计划加载的特征: ['lexicon_distance', 'phonology_distance', 'syntax_distance', 'overall_distance']
成功加载 4 个特征。
正在从文件 'Data3/info.npz' 加载数据...
计划加载的特征: ['coords']
成功加载 1 个特征。


# **Data 1**

In [4]:
dict_ = load_feats(name='Data1', type='info')

正在从文件 'Data1/data.pkl' 加载数据...
计划加载的特征: ['word', 'initial', 'final_1', 'final_2', 'final_3', 'tone']
成功加载 6 个特征。


In [ ]:
import pickle

# 加载数据
with open('Data1/data.pkl', 'rb') as f:
    data_dict = pickle.load(f)

# 使用数据
word_list = data_dict['word']
initial_list = data_dict['initial']
final_1_list = data_dict['final_1']
final_2_list = data_dict['final_2']
final_3_list = data_dict['final_3']
tone_list = data_dict['tone']


['多',
 '拖',
 '驼',
 '挪',
 '锣',
 '箩',
 '搓',
 '歌',
 '哥',
 '蛾',
 '鹅',
 '俄',
 '河',
 '何',
 '阿',
 '舵',
 '哪',
 '左',
 '可',
 '我',
 '那',
 '佐',
 '个',
 '饿',
 '贺',
 '茄',
 '波',
 '菠',
 '颇',
 '坡',
 '玻',
 '婆',
 '魔',
 '摩',
 '馍',
 '骡',
 '螺',
 '脶',
 '矬',
 '蓑',
 '梭',
 '嗦',
 '莎',
 '锅',
 '戈',
 '科',
 '窠',
 '棵',
 '讹',
 '禾',
 '倭',
 '踒',
 '窝',
 '跛',
 '朵',
 '妥',
 '椭',
 '惰',
 '垛',
 '裸',
 '瘰',
 '坐',
 '锁',
 '琐',
 '果',
 '裹',
 '馃',
 '颗',
 '火',
 '伙',
 '祸',
 '破',
 '剁',
 '唾',
 '糯',
 '摞',
 '锉',
 '座',
 '课',
 '卧',
 '货',
 '涴',
 '瘸',
 '膀',
 '巴',
 '芭',
 '疤',
 '爬',
 '琶',
 '杷',
 '钯',
 '麻',
 '痲',
 '蟆',
 '妈',
 '拿',
 '茶',
 '搽',
 '渣',
 '叉',
 '杈',
 '茬',
 '沙',
 '纱',
 '加',
 '痂',
 '嘉',
 '牙',
 '芽',
 '衙',
 '伢',
 '哈',
 '霞',
 '瑕',
 '遐',
 '鸦',
 '丫',
 '桠',
 '马',
 '码',
 '贾',
 '雅',
 '哑',
 '霸',
 '爸',
 '怕',
 '帕',
 '耙',
 '骂',
 '诈',
 '榨',
 '岔',
 '乍',
 '架',
 '驾',
 '嫁',
 '稼',
 '价',
 '揢',
 '砑',
 '暇',
 '亚',
 '些',
 '邪',
 '爹',
 '遮',
 '蛇',
 '奢',
 '赊',
 '佘',
 '耶',
 '爷',
 '姐',
 '且',
 '写',
 '者',
 '扯',
 '社',
 '惹',
 '也',
 '野',
 '借',
 '泻',
 '卸',
 '谢',
 '蔗',
 '麝'

# **Data 2**

In [16]:
dict_ = load_feats(name='Data2', type='mfcc_dialect_mean')
dict_ = load_feats(name='Data2', type='mfcc_slice_mean')
dict_ = load_feats(name='Data2', type='mfcc_dialect_gmm_ivector')

正在从文件 'Data2/dialect_mean_features.npz' 加载数据...
计划加载的特征: ['features', 'names']
成功加载 2 个特征。
正在从文件 'Data2/dialect_slice_mean_features.npz' 加载数据...
计划加载的特征: ['features', 'names']
成功加载 2 个特征。
正在从文件 'Data2/mfcc_gmm_ivectordialect.npz' 加载数据...
计划加载的特征: ['features', 'names']
成功加载 2 个特征。


In [ ]:
import numpy as np

# 假设您的 .npz 文件名为 'my_data.npz'
# file_path = 'Data2/dialect_mean_features.npz' # 替换为您的实际文件路径
# 文件中包含的数组（键）: ['features', 'dialect_names']

# 遍历文件中的所有数组:
#   键名: features, 形状: (17, 39), 数据类型: float32
#   键名: dialect_names, 形状: (17,), 数据类型: <U5

# file_path = 'Data2/dialect_slice_mean_features.npz'
# 文件中包含的数组（键）: ['features', 'slice_names']
# 遍历文件中的所有数组:
#   键名: features, 形状: (77, 39), 数据类型: float32
#   键名: slice_names, 形状: (77,), 数据类型: <U3

# file_path = 'Data2/mfcc_gmm_ivectordialect.npz'
# 文件中包含的数组（键）: ['features', 'dialect_names']

# 遍历文件中的所有数组:
#   键名: features, 形状: (17, 400), 数据类型: float64
#   键名: dialect_names, 形状: (17,), 数据类型: <U5

try:
    # 使用 numpy.load() 函数加载 .npz 文件
    data = np.load(file_path)

    # data 对象类似于一个字典，您可以通过键来访问存储在里面的数组
    # 首先，可以查看文件中包含哪些数组（键）
    print("文件中包含的数组（键）:", data.files)

    # 然后，通过键名来访问具体的数组
    # 例如，如果文件中有一个名为 'arr_0' 的数组：
    if 'arr_0' in data:
        array_0 = data['arr_0']
        print("\n数组 'arr_0' 的内容:")
        print(array_0)
        print(f"数组 'arr_0' 的形状: {array_0.shape}")

    # 如果有其他命名的数组，比如 'my_array'：
    if 'my_array' in data:
        my_named_array = data['my_array']
        print("\n数组 'my_array' 的内容:")
        print(my_named_array)
        print(f"数组 'my_array' 的形状: {my_named_array.shape}")

    # 遍历所有数组并打印它们的名字和形状
    print("\n遍历文件中的所有数组:")
    for key in data.files:
        array_data = data[key]
        print(f"  键名: {key}, 形状: {array_data.shape}, 数据类型: {array_data.dtype}")

    # 使用完毕后，如果您不需要再访问 data 对象，可以关闭它（虽然对于 .npz，通常不是严格必须的，
    # 因为数据已经加载到内存，但这是一个好习惯，特别是对于其他类型的文件对象）
    data.close()

except FileNotFoundError:
    print(f"错误：文件 {file_path} 未找到。")
except Exception as e:
    print(f"读取文件时发生错误: {e}")

文件中包含的数组（键）: ['features', 'dialect_names']

遍历文件中的所有数组:
  键名: features, 形状: (17, 400), 数据类型: float64
  键名: dialect_names, 形状: (17,), 数据类型: <U5


# End